<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#download-jar-files-and-sparkxgb.zip-file" data-toc-modified-id="download-jar-files-and-sparkxgb.zip-file-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>download jar files and sparkxgb.zip file</a></span></li></ul></div>

# download jar files and sparkxgb.zip file
- https://mvnrepository.com/artifact/ml.dmlc/xgboost4j/0.72
- https://mvnrepository.com/artifact/ml.dmlc/xgboost4j-spark/0.72
- https://github.com/dmlc/xgboost/files/2161553/sparkxgb.zip

In [1]:
import xgboost
print(xgboost.__version__)

0.90


In [2]:
import os

os.environ['PYSPARK_SUBMIT_ARGS'] ='--jars xgboost4j-spark-0.90.jar,xgboost4j-0.90.jar pyspark-shell'

HOME = os.path.expanduser('~')

import findspark
# findspark.init(HOME + "/Softwares/spark-3.0.0-bin-hadoop2.7")
findspark.init(HOME + "/Softwares/Spark/spark-2.4.6-bin-hadoop2.7")


import pyspark
from pyspark.sql.session import SparkSession
from pyspark.sql.types import *
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.sql.functions import col

spark = SparkSession\
        .builder\
        .appName("PySpark XGBOOST Titanic")\
        .getOrCreate()

spark.sparkContext.addPyFile(HOME + "/Softwares/Spark/pyspark-xgboost_0.90.zip")

print(pyspark.__version__)

2.4.6


In [3]:
from sparkxgb import XGBoostClassifier

In [4]:
# XGBoostClassifier?

In [5]:
schema = StructType(
  [StructField("PassengerId", DoubleType()),
    StructField("Survival", DoubleType()),
    StructField("Pclass", DoubleType()),
    StructField("Name", StringType()),
    StructField("Sex", StringType()),
    StructField("Age", DoubleType()),
    StructField("SibSp", DoubleType()),
    StructField("Parch", DoubleType()),
    StructField("Ticket", StringType()),
    StructField("Fare", DoubleType()),
    StructField("Cabin", StringType()),
    StructField("Embarked", StringType())
  ])

In [6]:
df_raw = spark\
  .read\
  .option("header", "true")\
  .schema(schema)\
  .csv(HOME + "/Datasets/kaggle/titanic/titanic_train.csv")

In [7]:
df_raw.show(2)

+-----------+--------+------+--------------------+------+----+-----+-----+---------+-------+-----+--------+
|PassengerId|Survival|Pclass|                Name|   Sex| Age|SibSp|Parch|   Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+---------+-------+-----+--------+
|        1.0|     0.0|   3.0|Braund, Mr. Owen ...|  male|22.0|  1.0|  0.0|A/5 21171|   7.25| null|       S|
|        2.0|     1.0|   1.0|Cumings, Mrs. Joh...|female|38.0|  1.0|  0.0| PC 17599|71.2833|  C85|       C|
+-----------+--------+------+--------------------+------+----+-----+-----+---------+-------+-----+--------+
only showing top 2 rows



In [8]:
df = df_raw.na.fill(0)

In [9]:
sexIndexer = StringIndexer()\
  .setInputCol("Sex")\
  .setOutputCol("SexIndex")\
  .setHandleInvalid("keep")
    
cabinIndexer = StringIndexer()\
  .setInputCol("Cabin")\
  .setOutputCol("CabinIndex")\
  .setHandleInvalid("keep")
    
embarkedIndexer = StringIndexer()\
  .setInputCol("Embarked")\
  .setOutputCol("EmbarkedIndex")\
  .setHandleInvalid("keep")

In [10]:
vectorAssembler = VectorAssembler()\
  .setInputCols(["Pclass", "SexIndex", "Age", "SibSp", "Parch", "Fare", "CabinIndex", "EmbarkedIndex"])\
  .setOutputCol("features")

In [11]:
model = XGBoostClassifier(
    featuresCol="features", 
    labelCol="Survival", 
    predictionCol="prediction"
)

# pipeline = Pipeline().setStages([sexIndexer, cabinIndexer, embarkedIndexer, vectorAssembler, model])
# trainDF, testDF = df.randomSplit([0.8, 0.2], seed=24)
# model = pipeline.fit(trainDF)

TypeError: 'JavaPackage' object is not callable

In [ ]:
from pyspark.ml.classification import RandomForestClassifier

model = RandomForestClassifier(
    featuresCol="features", 
    labelCol="Survival", 
    predictionCol="prediction"
)

pipeline = Pipeline().setStages([sexIndexer, cabinIndexer, embarkedIndexer, vectorAssembler, model])
trainDF, testDF = df.randomSplit([0.8, 0.2], seed=24)
model = pipeline.fit(trainDF)

In [ ]:
model.transform(testDF).select(col("PassengerId"), col("prediction")).show()